<a href="https://colab.research.google.com/github/LuisIrigoyen/trabajo_final_RL/blob/main/proyecto_final_APR_V06_3_en_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False


## 1.3. Montar carpeta de datos local (solo Colab)

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['dqn_models', 'checkpoints', 'dqn_05_weights.h5f.data-00000-of-00001', 'dqn_05_weights.h5f.index', 'dqn_06_weights.h5f.index', 'dqn_06_weights.h5f.data-00000-of-00001', 'dqn_07_weights.h5f.index', 'dqn_07_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.index', '.ipynb_checkpoints', 'dqn_09_weights.h5f.index', 'dqn_09_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.index', 'checkpoint']


## 1.4. Instalar librerías necesarias

In [3]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12  #2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.05
  %pip install Keras==2.2.4
  %pip install tensorflow==2.1.0
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-fxv5jt36
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-fxv5jt36
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


## **PARTE 3**. Desarrollo y preguntas
Importar librerías

In [4]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D
#from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint


#### Configuración base

In [6]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

#--- nuevo -- ejfm
height, width, channels = env.observation_space.shape
obs = env.reset()
print(height, width, channels)
print("Forma de la observación:", obs.shape)  # Debe ser (height, width, channels)
env.unwrapped.get_action_meanings()

210 160 3
Forma de la observación: (210, 160, 3)


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [26]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):

        if batch.ndim == 4:
            batch = np.expand_dims(batch, axis=-1)
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

## 1. EJFM Implementación de la red neuronal

In [27]:
def build_model(height, width, channels, actions):

    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(4,84,84,1)))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

In [28]:
model = build_model(84, 84, 1, nb_actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 4, 20, 20, 32)     2080      
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 9, 9, 64)       32832     
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 7, 7, 64)       36928     
                                                                 
 flatten_1 (Flatten)         (None, 12544)             0         
                                                                 
 dense_5 (Dense)             (None, 512)               6423040   
                                                                 
 dense_6 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 6,497,958
Trainable params: 6,497,958
No

## 2. EJFM Implementación de la solución DQN

In [29]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [30]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(),
        attr='eps',
        value_max=1.0,
        value_min=0.1,
        value_test=0.005,
        nb_steps=200000)
    memory = SequentialMemory(limit=100000, window_length=4)
    dqn = DQNAgent(model=model,
                   memory=memory,
                   policy=policy,
                   enable_dueling_network=True,
                   dueling_type='avg',
                   nb_actions=nb_actions,
                   batch_size=32,
                   target_model_update=1000,
                   processor=AtariProcessor(),
                   nb_steps_warmup=10000
                  )
    return dqn

In [31]:
from rl.callbacks import ModelIntervalCheckpoint

# Guardar el modelo cada 10,000 pasos
checkpoint_callback = ModelIntervalCheckpoint(
    filepath='checkpoints/dqn_v06_3_weights_{step}.h5f',
    interval=10000,
    verbose=1
)

In [32]:
import os
os.makedirs("checkpoints", exist_ok=True)

In [33]:
from keras.optimizers import Adam

dqn = build_agent(model, nb_actions)
dqn.compile(Adam(learning_rate=0.00025, epsilon=1e-4))


In [34]:
from datetime import datetime

print(f"⏱ Tiempo de inicio: {datetime.now().strftime('%H:%M:%S')}")

dqn.fit(env, nb_steps=300000, visualize=False, verbose=2,callbacks=[checkpoint_callback])

print(f"⏱ Tiempo de fin: {datetime.now().strftime('%H:%M:%S')}")

# Guardando modelo
dqn.save_weights('dqn_06_3_weights.h5f')


⏱ Tiempo de inicio: 16:59:53
Training for 300000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    476/300000: episode: 1, duration: 2.834s, episode steps: 476, steps per second: 168, episode reward:  4.000, mean reward:  0.008 [ 0.000,  1.000], mean action: 2.433 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   1501/300000: episode: 2, duration: 4.938s, episode steps: 1025, steps per second: 208, episode reward: 19.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 2.448 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   1868/300000: episode: 3, duration: 1.793s, episode steps: 367, steps per second: 205, episode reward:  2.000, mean reward:  0.005 [ 0.000,  1.000], mean action: 2.444 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2716/300000: episode: 4, duration: 4.024s, episode steps: 848, steps per second: 211, episode reward: 16.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 2.522 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   3110/300000: episode: 5, duration: 1.908s, episode steps: 394, steps per second: 206, episode reward:  6.000

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  10131/300000: episode: 15, duration: 18.110s, episode steps: 437, steps per second:  24, episode reward:  3.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 2.579 [0.000, 5.000],  loss: 0.005750, mean_q: 0.042665, mean_eps: 0.954705
  11216/300000: episode: 16, duration: 129.978s, episode steps: 1085, steps per second:   8, episode reward: 13.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.495 [0.000, 5.000],  loss: 0.006656, mean_q: 0.053420, mean_eps: 0.951971
  11732/300000: episode: 17, duration: 62.055s, episode steps: 516, steps per second:   8, episode reward:  8.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.597 [0.000, 5.000],  loss: 0.007587, mean_q: 0.082515, mean_eps: 0.948369
  12563/300000: episode: 18, duration: 99.186s, episode steps: 831, steps per second:   8, episode reward: 11.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.526 [0.000, 5.000],  loss: 0.006488, mean_q: 0.099772, mean_eps: 0.945338
  13189/300000: episode: 19, d

In [92]:
#Ejecutando a partir de un callbacks
env.close()  # Asegúrate de cerrar entorno anterior
env = gym.make("SpaceInvaders-v0")


In [89]:
# Construir modelo y agente exactamente como en el entrenamiento
model = build_model(84, 84, 1, nb_actions)  # Usa tu propia función build_model
dqn = build_agent(model, nb_actions)        # Usa tu propia función build_agent

# Compilar el agente (necesario antes de usarlo)
from keras.optimizers import Adam
dqn.compile(Adam(learning_rate=0.00025, epsilon=1e-4))

# Cargar los pesos desde el checkpoint
dqn.load_weights("checkpoints/dqn_v06_3_weights_90000.h5f")

# Continuar entrenamiento
from datetime import datetime
print(f"⏱ Inicio nuevo entrenamiento: {datetime.now().strftime('%H:%M:%S')}")

dqn.fit(env, nb_steps=120000, visualize=False, verbose=2, callbacks=[checkpoint_callback])

print(f"⏱ Fin nuevo entrenamiento: {datetime.now().strftime('%H:%M:%S')}")


⏱ Inicio nuevo entrenamiento: 21:30:22
Training for 120000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    845/120000: episode: 1, duration: 5.069s, episode steps: 845, steps per second: 167, episode reward: 10.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.399 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   1574/120000: episode: 2, duration: 3.993s, episode steps: 729, steps per second: 183, episode reward:  6.000, mean reward:  0.008 [ 0.000,  1.000], mean action: 2.432 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2344/120000: episode: 3, duration: 4.252s, episode steps: 770, steps per second: 181, episode reward: 11.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.409 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2816/120000: episode: 4, duration: 2.642s, episode steps: 472, steps per second: 179, episode reward:  7.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.475 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   3593/120000: episode: 5, duration: 4.261s, episode steps: 777, steps per second: 182, episode reward: 11.000,

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  10028/120000: episode: 13, duration: 9.388s, episode steps: 615, steps per second:  66, episode reward:  8.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.494 [0.000, 5.000],  loss: 0.041838, mean_q: 5.543542, mean_eps: 0.954937
  10984/120000: episode: 14, duration: 119.720s, episode steps: 956, steps per second:   8, episode reward: 13.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.492 [0.000, 5.000],  loss: 0.035422, mean_q: 5.479570, mean_eps: 0.952725
  11363/120000: episode: 15, duration: 47.558s, episode steps: 379, steps per second:   8, episode reward:  5.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.430 [0.000, 5.000],  loss: 0.033018, mean_q: 5.519955, mean_eps: 0.949722
  11987/120000: episode: 16, duration: 78.071s, episode steps: 624, steps per second:   8, episode reward:  7.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.462 [0.000, 5.000],  loss: 0.033763, mean_q: 5.539098, mean_eps: 0.947465
  12467/120000: episode: 17, dur

In [91]:
# Testing part to calculate the mean reward
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
weights_filename = 'dqn_06_3_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 11.000, steps: 625
Episode 2: reward: 6.000, steps: 520
Episode 3: reward: 8.000, steps: 407
Episode 4: reward: 10.000, steps: 887
Episode 5: reward: 15.000, steps: 779
Episode 6: reward: 18.000, steps: 1118
Episode 7: reward: 24.000, steps: 1295
Episode 8: reward: 7.000, steps: 732
Episode 9: reward: 8.000, steps: 454
Episode 10: reward: 19.000, steps: 806


In [46]:
from google.colab import drive
import shutil
import glob  # <-- ¡Falta este import!
import os

# 1. Montar Google Drive
#drive.mount('/content/drive')

# 2. Crear directorio en Drive si no existe
os.makedirs('/content/gdrive/MyDrive/Colab_Outputs', exist_ok=True)

# 3. Buscar y copiar todos los archivos coincidentes
archivos = glob.glob('dqn_06_3_weights*')  # Encuentra todos los archivos con ese patrón

for archivo in archivos:
    shutil.copy(
        archivo,
        f'/content/gdrive/MyDrive/Colab_Outputs/{os.path.basename(archivo)}'
    )
    print(f"¡Archivo {archivo} copiado a Drive!")

# 4. Buscar y copiar todos los archivos coincidentes
archivos = glob.glob('checkpoints/dqn_v06_3_weights*')  # Encuentra todos los archivos con ese patrón

for archivo in archivos:
    shutil.copy(
        archivo,
        f'/content/gdrive/MyDrive/Colab_Outputs/{os.path.basename(archivo)}'
    )

print("✅ ¡Todos los archivos copiados!")

¡Archivo dqn_06_3_weights.h5f.data-00000-of-00001 copiado a Drive!
¡Archivo dqn_06_3_weights.h5f.index copiado a Drive!
✅ ¡Todos los archivos copiados!
